In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

ModuleNotFoundError: ignored

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Test.csv to Test.csv
Saving Train.csv to Train.csv


In [0]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

In [7]:

class NewTrain():
    def __init__(self):
      train_X = train.drop("Result",axis=1).as_matrix().reshape(-1,1,90,90)
      train_Y = train.Result.as_matrix().tolist()
      self.datalist = train_X
      self.labellist = train_Y
    def __getitem__(self, index):
        return torch.Tensor(self.datalist[index].astype(float)), self.labellist[index]

    def __len__(self):
        return self.datalist.shape[0]
      

    
train_data=NewTrain()
print(len(train_data))
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=1, shuffle=True,num_workers=2)
print(len(train_loader))     

100
100


In [8]:
class NewTest():
    def __init__(self):
      test_X = test.drop("Result",axis=1).as_matrix().reshape(-1,1,90,90)
      test_Y = test.Result.as_matrix().tolist()
      self.datalist = test_X
      self.labellist = test_Y
    def __getitem__(self, index):
        return torch.Tensor(self.datalist[index].astype(float)), self.labellist[index]

    def __len__(self):
        return self.datalist.shape[0]

test_data=NewTest()
print(len(test_data))
test_loader = torch.utils.data.DataLoader( dataset=test_data,batch_size=1, shuffle=True)
print(len(test_loader))

45
45


In [0]:
class Pro(nn.Module):
    def __init__(self):
        super(Pro, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=7)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=7)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(6480, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 6480)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
      
def train(model,train_loader,optimizer,epoch):
    model.train()
    train_loss=0
    for batch_idx, (inputs,target) in enumerate(train_loader):
      inputs,target = torch.autograd.Variable(inputs), torch.autograd.Variable(target)
      optimizer.zero_grad()
      output=model(inputs)
      loss =criterion(output.double(),target.long())
      loss.backward()
      optimizer.step()
      train_loss+=loss.data[0]
      if batch_idx % 1 ==0:
        print ('Epoch [%d/%d],  Loss: %.4f' %(epoch+1, 5, loss.data[0]))
        train_loss=0
        
def test(model,test_loader):
    model.eval()
    test_loss=0
    correct=0
    for (inputs,target) in test_loader:
        inputs,target=torch.autograd.Variable(inputs), torch.autograd.Variable(target)
        output=model(inputs)
        print(output)
        loss =criterion(output.double(),target.long())
        test_loss +=loss.data[0] # sum up batch loss
        pred =  np.argmax(output.data.cpu().numpy(), axis=1)
        #output.max()[1].double() # get the index of the max log-probability
        label=target.data.cpu().numpy()
        correct += (pred ==label).sum()
        test_loss /= len(test_loader.dataset)
        
    print("The accuracy percentage is {}".format((correct/50)*100))
    print('\n')
    print("The average loss is{}".format(test_loss))
    

In [10]:
model = Pro()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(5):
  train(model, train_loader, optimizer, epoch)
test(model, test_loader)

Epoch [1/5],  Loss: 0.6610
Epoch [1/5],  Loss: 0.5662
Epoch [1/5],  Loss: 1.0183
Epoch [1/5],  Loss: 0.7698
Epoch [1/5],  Loss: 0.7132
Epoch [1/5],  Loss: 0.7308
Epoch [1/5],  Loss: 0.8499
Epoch [1/5],  Loss: 0.6917
Epoch [1/5],  Loss: 0.7018
Epoch [1/5],  Loss: 0.7118
Epoch [1/5],  Loss: 0.6568
Epoch [1/5],  Loss: 0.6627
Epoch [1/5],  Loss: 0.6715
Epoch [1/5],  Loss: 0.5960
Epoch [1/5],  Loss: 0.6223
Epoch [1/5],  Loss: 0.9057
Epoch [1/5],  Loss: 0.8146
Epoch [1/5],  Loss: 0.5490
Epoch [1/5],  Loss: 0.5545
Epoch [1/5],  Loss: 0.7494
Epoch [1/5],  Loss: 0.4101
Epoch [1/5],  Loss: 0.7499
Epoch [1/5],  Loss: 0.7288
Epoch [1/5],  Loss: 1.3355
Epoch [1/5],  Loss: 0.6029
Epoch [1/5],  Loss: 0.6566
Epoch [1/5],  Loss: 0.8160
Epoch [1/5],  Loss: 0.6619
Epoch [1/5],  Loss: 0.7337
Epoch [1/5],  Loss: 0.8234
Epoch [1/5],  Loss: 0.7077
Epoch [1/5],  Loss: 0.6722
Epoch [1/5],  Loss: 0.7180
Epoch [1/5],  Loss: 0.7255
Epoch [1/5],  Loss: 0.6519
Epoch [1/5],  Loss: 0.6888
Epoch [1/5],  Loss: 0.6761
E

Epoch [3/5],  Loss: 0.6958
Epoch [3/5],  Loss: 0.6955
Epoch [3/5],  Loss: 0.6939
Epoch [3/5],  Loss: 0.6978
Epoch [3/5],  Loss: 0.6818
Epoch [3/5],  Loss: 0.7040
Epoch [3/5],  Loss: 0.7145
Epoch [3/5],  Loss: 0.6947
Epoch [3/5],  Loss: 0.7003
Epoch [3/5],  Loss: 0.6834
Epoch [3/5],  Loss: 0.6810
Epoch [3/5],  Loss: 0.7062
Epoch [3/5],  Loss: 0.7112
Epoch [3/5],  Loss: 0.6719
Epoch [3/5],  Loss: 0.7063
Epoch [3/5],  Loss: 0.6824
Epoch [3/5],  Loss: 0.6803
Epoch [3/5],  Loss: 0.7184
Epoch [3/5],  Loss: 0.6658
Epoch [3/5],  Loss: 0.7146
Epoch [3/5],  Loss: 0.6763
Epoch [3/5],  Loss: 0.6708
Epoch [3/5],  Loss: 0.7259
Epoch [3/5],  Loss: 0.6620
Epoch [3/5],  Loss: 0.7273
Epoch [3/5],  Loss: 0.6597
Epoch [3/5],  Loss: 0.7262
Epoch [3/5],  Loss: 0.7294
Epoch [3/5],  Loss: 0.7195
Epoch [3/5],  Loss: 0.7012
Epoch [3/5],  Loss: 0.6886
Epoch [3/5],  Loss: 0.6924
Epoch [3/5],  Loss: 0.6944
Epoch [3/5],  Loss: 0.7072
Epoch [3/5],  Loss: 0.6788
Epoch [3/5],  Loss: 0.7179
Epoch [3/5],  Loss: 0.6750
E

Epoch [5/5],  Loss: 0.7170
Epoch [5/5],  Loss: 0.6721
Epoch [5/5],  Loss: 0.7205
Epoch [5/5],  Loss: 0.6685
Epoch [5/5],  Loss: 0.6675
Epoch [5/5],  Loss: 0.7171
Epoch [5/5],  Loss: 0.7179
Epoch [5/5],  Loss: 0.7007
Epoch [5/5],  Loss: 0.6947
Epoch [5/5],  Loss: 0.6729
Epoch [5/5],  Loss: 0.6787
Epoch [5/5],  Loss: 0.6794
Epoch [5/5],  Loss: 0.7024
Epoch [5/5],  Loss: 0.6611
Epoch [5/5],  Loss: 0.6741
Epoch [5/5],  Loss: 0.6032
Epoch [5/5],  Loss: 0.5746
Epoch [5/5],  Loss: 0.7390
Epoch [5/5],  Loss: 0.6766
Epoch [5/5],  Loss: 0.7046
Epoch [5/5],  Loss: 0.6790
Epoch [5/5],  Loss: 0.6862
Epoch [5/5],  Loss: 0.6694
Epoch [5/5],  Loss: 0.6791
Epoch [5/5],  Loss: 0.7072
Epoch [5/5],  Loss: 1.0267
Epoch [5/5],  Loss: 0.7042
Epoch [5/5],  Loss: 0.7476
Epoch [5/5],  Loss: 0.6300
Epoch [5/5],  Loss: 0.5619
Epoch [5/5],  Loss: 0.7704
Epoch [5/5],  Loss: 0.7804
Epoch [5/5],  Loss: 0.6275
Epoch [5/5],  Loss: 0.8004
Epoch [5/5],  Loss: 0.6339
Epoch [5/5],  Loss: 0.6561
Epoch [5/5],  Loss: 0.5892
E